In [1]:
from datasets import load_from_disk
import pandas as pd

In [ ]:
! mkdir ../data
! wget https://storage.googleapis.com/usc-data/all-coref-resolved.tar.gz
! wget https://storage.googleapis.com/usc-data/full-source-scored-data.jsonl.gz
! mv all-coref-resolved.tar.gz ../data
! mv full-source-scored-data.jsonl.gz ../data
! tar -xvzf ../data/all-coref-resolved.tar.gz

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
data_dir = '../data'
data_dir = '../../../bloomberg-research/press-releases/data/s_p_500_backlinks'
source_df = pd.read_json(f'{data_dir}/full-source-scored-data.jsonl.gz', lines=True, compression='gzip', nrows=100)

In [3]:
article_d = load_from_disk(data_dir + '/all-coref-resolved')

Loading dataset from disk:   0%|          | 0/41 [00:00<?, ?it/s]

In [4]:
filtered_article_d = article_d.filter(lambda x: x['article_url'] in set(source_df['article_url']), num_proc=10)

In [7]:
#experimentation, please ignore
print(type(filtered_article_d))
articles = filtered_article_d.to_pandas()


sentences_with_quotes = (
    filtered_article_d
         .to_pandas()
         .merge(source_df, on='article_url'))

print(sentences_with_quotes.columns)
# for i in range(100):
    # print(sentences_with_quotes.iloc[i]['article_url'])


<class 'datasets.arrow_dataset.Dataset'>
Index(['article_url', 'target_timestamp_key', 'target_timestamp_x',
       'sort_criteria', 'wayback_url', 'wayback_timestamp', 'method',
       'links_x', 'article_text', 'word_lists', 'sent_lists', 'best_class_x',
       'coref_resolved_sents', 'links_y', 'target_timestamp_y', 'attributions',
       'best_class_y', 'quote_type'],
      dtype='object')


In [9]:
sentences_with_quotes

,article_url,target_timestamp_key,target_timestamp_x,sort_criteria,wayback_url,wayback_timestamp,method,links_x,article_text,word_lists,sent_lists,best_class_x,coref_resolved_sents,links_y,target_timestamp_y,attributions,best_class_y,quote_type
0,www.bbc.co.uk/blogs/researchanddevelopment/201...,target_timestamp,1.689984e+12,past_then_future,https://web.archive.org/web/20220627125011/www...,20220627125011,wayback,"[{'char_end_idx': 38, 'char_start_idx': 0, 'hr...",British Broadcasting CorporationHome \n Skip ...,"[[The, technology, was, well, recieved, ,, but...","[The technology was well recieved, but it's ab...","[Distant_Evaluation, Distant_Evaluation, Dista...","[The technology was well recieved, but it's ab...","[{'char_end_idx': 38, 'char_start_idx': 0, 'hr...",1689984000000,"[journalist, journalist, journalist, BBC R&amp...","[Distant_Evaluation, Distant_Evaluation, Dista...","[No Quote, No Quote, No Quote, Direct Quote, N..."
1,www.prweb.com/releases/2015/02/prweb12528912.htm,target_timestamp,1.553818e+12,past_then_future,https://web.archive.org/web/20180707025122/www...,20180707025122,wayback,"[{'char_end_idx': 7, 'char_start_idx': 0, 'hre...",PRWeb \n Home \n News Center \n Blog \n Login...,"[[Based, on, a, cohesive, and, committed, phil...",[Based on a cohesive and committed philanthrop...,"[Distant_Evaluation, Distant_Expectations_Cons...",[Based on a cohesive and committed philanthrop...,"[{'char_end_idx': 7, 'char_start_idx': 0, 'hre...",1553817600000,"[Socrata Foundation, Socrata Foundation, Socra...","[Distant_Evaluation, Distant_Expectations_Cons...","[Background/Narrative, Background/Narrative, D..."
2,www.usatoday.com/story/news/health/2015/08/06/...,target_timestamp,1.521590e+12,past_then_future,https://web.archive.org/web/20170116111155/www...,20170116111155,wayback,"[{'char_end_idx': 2, 'char_start_idx': 0, 'hre...",SUBSCRIBE NOWto get home delivery \nGet the n...,"[[Post, to, FacebookCVS, drops, Viagra, ,, oth...","[Post to FacebookCVS drops Viagra, other drugs...","[Cause_General, Cause_General, Cause_General, ...","[Post to FacebookCVS drops Viagra, other drugs...","[{'char_end_idx': 2, 'char_start_idx': 0, 'hre...",1521590400000,"[CVS, CVS drops Viagra, other drugs, from insu...","[Cause_General, Cause_General, Cause_General, ...","[Statement/Public Speech, No Quote, Published ..."
3,thehill.com/changing-america/well-being/mental...,target_timestamp,1.671149e+12,past_then_future,https://web.archive.org/web/20221007232303/the...,20221007232303,wayback,"[{'char_end_idx': 17, 'char_start_idx': 0, 'hr...",Skip to content \n Well-Being \n Mental Healt...,"[[ , Skip, to, content, \n , Well, -, Being, \...",[ Skip to content \n Well-Being \n Mental Heal...,"[Cause_General, Cause_General, Cause_General, ...",[ Skip to content \n Well-Being \n Mental Heal...,"[{'char_end_idx': 17, 'char_start_idx': 0, 'hr...",1671148800000,[researchers set out to determine who teens tu...,"[Cause_General, Cause_General, Cause_General, ...","[Background/Narrative, Published Work/Press Re..."
4,www.alternet.org/author/danielle-ivory,target_timestamp,1.674346e+12,past_then_future,https://web.archive.org/web/20220703174543/www...,20220703174543,wayback,"[{'char_end_idx': 2, 'char_start_idx': 0, 'hre...",news &amp; politics \n trump \n the right win...,"[[\n , The, Right, Wing, \n , Religious, right...",[\n The Right Wing \n Religious right \n GO AD...,"[Distant_Evaluation, Cause_General, Distant_Ev...",[\n The Right Wing \n Religious right \n GO AD...,"[{'char_end_idx': 2, 'char_start_idx': 0, 'hre...",1674345600000,"[Tim Murtaugh, Trump, What little was publicly...","[Distant_Evaluation, Cause_General, Distant_Ev...","[No Quote, No Quote, No Quote, Direct Quote, D..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,patch.com/maryland/bethesda-chevychase/restaur...,target_timestamp,1.553818e+12,past_then_future,https://web.archive.org/web/20160319153910/pat...,20160319153910,wayback,"[{'char_end_idx'

In [ ]:
disallowed_quote_types = set(['Other', 'Background/Narrative', 'No Quote'])

sentences_with_quotes = (
    filtered_article_d
         .to_pandas()
         .merge(source_df, on='article_url')
         [['article_url', 'attributions', 'quote_type', 'sent_lists',]]
         .explode(['attributions', 'quote_type', 'sent_lists'])
)

sentences_with_quotes = (sentences_with_quotes
     .assign(attributions=lambda df:
             df.apply(lambda x: x['attributions'] if x['quote_type'] not in disallowed_quote_types else np.nan, axis=1)
    )
)

In [ ]:
one_article = (
    sentences_with_quotes
         .loc[lambda df: df['article_url'] == df['article_url'].unique()[1]]
        .reset_index(drop=True)
)

In [ ]:
doc_str = one_article[['sent_lists', 'attributions']].to_csv(sep='\t', index=False)
json_str = one_article[['sent_lists', 'attributions']].to_json(lines=True, orient='records')

I ran llama3 locally using a cool program called Ollama: https://ollama.com/. You can follow the link to install the program.

This can be useful for trying things out before we work out compute access.

Run this in your terminal:

`ollama run llama3`

Even llama3 8b is pretty powerful. Llama 70b is better, but that may not run on your local computer.

In [ ]:
import requests
import pyperclip

In [ ]:
pyperclip.copy(doc_str)
pyperclip.copy(json_str)

In [ ]:
r = requests.post(
    'http://localhost:11434/api/generate',
    json = {
        "model": "llama3",
        "prompt":f"""
            Here is a news article, with each sentence annotated according to the source of it's information:
            ```
            {json_str}
            ```

            Please summarize each of our source annotations. Tell me in one paragraph per source: (1) who the source is (2) what informational content they provide to the article.
            Only rely on the annotations I have provided, don't identify additional sources.
        """,
        "stream": False
})

In [ ]:
print(r.json()['response'])

In [16]:
a_to_pr_df = pd.read_csv('../../../bloomberg-research/press-releases/data/s_p_500_backlinks/article-to-pr-mapper.csv', index_col=0)

In [20]:
a_to_pr_df['Target URL'].value_counts().value_counts()

count
1      127076
2       21442
3        8404
4        4573
5        2934
        ...  
284         1
282         1
276         1
275         1
177         1
Name: count, Length: 343, dtype: int64

In [73]:
candidate_df = (
    a_to_pr_df
        .loc[lambda df: df['URL'].isin(df['URL'].value_counts().loc[lambda s: s == 1].index)]    
        .loc[lambda df: df['Target URL'].str.len() > 20]    
        .loc[lambda df: df['Target URL'].isin(df['Target URL'].value_counts().loc[lambda s: (s > 2) & (s < 100)].index)]
        # 
        # covered by multiple news outlets
        # 
        # .loc[lambda df: df['Target URL'].isin(
        #     df
        #          [['Target URL', 'news_url_domain']]
        #          .value_counts().unstack()
        #          .fillna(0).pipe(lambda df: (df > 0).astype(int))
        #          .sum(axis=1)
        #          .loc[lambda s: s > 1]
        #          .index
        # )]       
)

In [54]:
a_to_pr_df.shape

(826819, 6)

In [74]:
candidate_df.shape

(107654, 6)

In [55]:
candidate_df.shape

(120993, 6)

In [100]:
candidate_df.to_csv('cache/news_articles_covering_the_same_press_releases.csv')

In [101]:
! open .

In [ ]:
import pandas as pd 
candidate_df = pd.read_csv('cache/news_articles_covering_the_same_press_releases.csv')
pr_url_set = set(candidate_df['Target URL'].drop_duplicates().values)
filtered_pr_d = article_d.filter(lambda x: x['article_url'] in pr_url_set, num_proc=10)

In [89]:
filtered_pr_d

Dataset({
    features: ['article_url', 'target_timestamp_key', 'target_timestamp', 'sort_criteria', 'wayback_url', 'wayback_timestamp', 'method', 'links', 'article_text', 'word_lists', 'sent_lists', 'best_class', 'coref_resolved_sents'],
    num_rows: 9607
})

In [95]:
filtered_pr_df = filtered_pr_d.to_pandas()

In [98]:
filtered_pr_df.columns.tolist()

'article_url', 'target_timestamp_key', 'target_timestamp', 'article_text', 'sent_lists'

['article_url',
 'target_timestamp_key',
 'target_timestamp',
 'sort_criteria',
 'wayback_url',
 'wayback_timestamp',
 'method',
 'links',
 'article_text',
 'word_lists',
 'sent_lists',
 'best_class',
 'coref_resolved_sents']

In [99]:
filtered_pr_df

,article_url,target_timestamp_key,target_timestamp,sort_criteria,wayback_url,wayback_timestamp,method,links,article_text,word_lists,sent_lists,best_class,coref_resolved_sents
0,www.goldmansachs.com/media-relations/press-rel...,target_timestamp,1.497571e+12,past_then_future,https://web.archive.org/web/20170209232058/www...,20170209232058,wayback,"[{'char_end_idx': 15, 'char_start_idx': 6, 'hr...",Search\n twitter \n @GoldmanSachs \n @GSCareer...,"[[The, agreement, in, principle, will, resolve...",[The agreement in principle will resolve actua...,"[Distant_Expectations_Consequences, Distant_Ex...",[The agreement in principle will resolve actua...
1,www.generalmills.com/news/press-releases/gener...,target_timestamp,1.683245e+12,past_then_future,https://web.archive.org/web/20230503064152/www...,20230503064152,wayback,"[{'char_end_idx': 22, 'char_start_idx': 0, 'hr...",Skip to main content \n Stands for Good \nApr...,"[[MINNEAPOLIS, ,, MN, —, General, Mills, today...","[MINNEAPOLIS, MN — General Mills today announc...","[Cause_General, Cause_General, Cause_General, ...","[MINNEAPOLIS, MN — General Mills today announc..."
2,www.pfizer.com/news/press-release/press-releas...,target_timestamp,1.545178e+12,past_then_future,https://web.archive.org/web/20181213122729/www...,20181213122729,wayback,"[{'char_end_idx': 2, 'char_start_idx': 0, 'hre...",Careers \n Working Here \n Mission &amp; Purp...,"[[Biologics, have, revolutionized, the, treatm...",[Biologics have revolutionized the treatment o...,"[Cause_General, Distant_Evaluation, Distant_Ex...",[Biologics have revolutionized the treatment o...
3,www.pfizer.com/news/press-release/press-releas...,target_timestamp,1.657498e+12,past_then_future,https://web.archive.org/web/20220709173902/www...,20220709173902,wayback,"[{'char_end_idx': 22, 'char_start_idx': 0, 'hr...",Skip to main content \n Science \n Clinical T...,"[[Cases, continue, to, accumulate, according, ...",[Cases continue to accumulate according to the...,"[Cause_General, Distant_Historical, Distant_Ex...",[Cases continue to accumulate according to the...
4,about.netflix.com/en/news/free-educational-doc...,target_timestamp,1.611619e+12,past_then_future,https://web.archive.org/web/20201224130937/abo...,20201224130937,wayback,"[{'char_end_idx': 18, 'char_start_idx': 0, 'hr...",Back to All News \nResponding to teachers' re...,"[[ , Back, to, All, News, \n, Responding, to, ...",[ Back to All News \nResponding to teachers' r...,"[Cause_General, Distant_Evaluation, Cause_Gene...",[ Back to All News \nResponding to teachers' r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9602,mediaroom.wm.com/the-battle-against-recycling-...,target_timestamp,1.613866e+12,past_then_future,https://web.archive.org/web/20210124063917/med...,20210124063917,wayback,"[{'char_end_idx': 15, 'char_start_idx': 9, 'hr...",Site Menu\n Blog \n News Releases \n Social Me...,"[[Simply, put, ,, many, of, the, items, we, al...","[Simply put, many of the items we all want to ...","[Distant_Evaluation, Distant_Evaluation, Dista...","[Simply put, many of the items we all want to ..."
9603,www.salesforce.com/products/marketing-cloud/ov...,target_timestamp,1.496275e+12,past_then_future,https://web.archive.org/web/20170530030856/www...,20170530030856,wayback,"[{'char_end_idx': 12, 'char_start_idx': 0, 'hr...",Watch Demo \n Guided Tour \n Watch Demo \n Gu...,"[[ , Watch, Demo, \n , Guided, Tour, \n , Watc...",[ Watch Demo \n Guided Tour \n Watch Demo \n G...,"[Cause_General, Cause_General, Distant_Evaluat...",[ Watch Demo \n Guided Tour \n Watch Demo \n G...
9604,usa.visa.com/about-visa/newsroom/press-release...,target_timestamp,1.624838e+12,past_then_future,https://web.archive.org/web/20210614215115/usa...,20210614215115,wayback,"[{'char_end_idx': 17, 'char_start_idx': 0, 'hr...",Skip to content \nVisa Statement on Planned A...,"[[5, ,, 2020, -, -, Visa, Inc., (, NYSE, :, V,...","[5, 2020-- Visa Inc. (NYSE:V) issued the follo...","[Cause_General, Distant_Evaluatio

In [ ]:
article_d

In [62]:
filtered_article_d = article_d.filter(lambda x: x['article_url'] in set(candidate_df['URL']), num_proc=10)

Filter (num_proc=10):   0%|          | 0/496380 [00:00<?, ? examples/s]